In [12]:
import yfinance as yf
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import statsmodels.api as sm

sns.set(style = 'whitegrid')

import warnings
warnings.filterwarnings('ignore')

In [17]:
acoes = ['ITUB4.SA', 'BBDC4.SA', 'DIRR3.SA', 'JBSS3.SA', 'GUAR3.SA', '^BVSP']

ydata = yf.download(acoes, '2015-01-01', '2019-12-31', 'id')
dados = ydata['Adj Close']
dados = dados.rename(columns = {'^BVSP': 'IBOV'})
dados.head()

[*********************100%%**********************]  6 of 6 completed


Ticker,BBDC4.SA,DIRR3.SA,GUAR3.SA,ITUB4.SA,JBSS3.SA,IBOV
Date,,,,,,
2015-01-02,8.725244,4.704646,8.123262,11.964714,7.803239,48512.0
2015-01-05,8.743231,4.477847,7.866183,12.024843,7.840221,47517.0
2015-01-06,9.030851,4.349908,7.840375,12.219362,7.655311,48001.0
2015-01-07,9.389745,4.431324,7.984301,12.661441,7.869808,49463.0
2015-01-08,9.438101,4.512739,7.867176,12.859507,7.936375,49943.0


In [18]:
dados = dados * 100/ dados.iloc[0]
dados_chg = dados.pct_change()
dados_chg.fillna(0, inplace = True)
dados_chg.head()

Ticker,BBDC4.SA,DIRR3.SA,GUAR3.SA,ITUB4.SA,JBSS3.SA,IBOV
Date,,,,,,
2015-01-02,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2015-01-05,0.002062,-0.048208,-0.031647,0.005026,0.004739,-0.020510
2015-01-06,0.032896,-0.028571,-0.003281,0.016176,-0.023585,0.010186
2015-01-07,0.039741,0.018717,0.018357,0.036179,0.028019,0.030458
2015-01-08,0.005150,0.018373,-0.014669,0.015643,0.008459,0.009704


In [19]:
# criação de dataframe com o índice de mercado dos ativos livre de risco

indices_chg = dados_chg[['IBOV']]

tlr_anual = 0.12 # estimativa selic
tlr_diaria = (1 + tlr_anual)**(1/252) - 1
indices_chg.insert(1, 'SELIC', tlr_diaria)
indices_chg.head()

Ticker,IBOV,SELIC
Date,,
2015-01-02,0.000000,0.00045
2015-01-05,-0.020510,0.00045
2015-01-06,0.010186,0.00045
2015-01-07,0.030458,0.00045
2015-01-08,0.009704,0.00045


In [20]:
dados.drop(['IBOV'], axis = 'columns', inplace = True)
dados_chg.drop(['IBOV'], axis = 'columns', inplace = True)

In [23]:
def calc_capm(ativo_chg, alr_chg, merc_chg): # ativo escolhido, ativo livre de risco e mercado

    ativo_m_alr = ativo_chg - alr_chg
    merc_m_alr = merc_chg - alr_chg

# transformar em vetores
    ativo_np = ativo_m_alr.to_numpy()
    merc_np = merc_m_alr.to_numpy()

# regressão linear
    merc_np2 = sm.add_constant(merc_np)
    model = sm.OLS(ativo_np, merc_np2)
    res = model.fit()

    return res


In [24]:
port_pesos = [0.1666, 0.1666, 0.1666, 0.1666, 0.1666]
port = dados.dot(port_pesos).pct_change()
port.fillna(0, inplace =  True)
res = calc_capm(port, indices_chg['SELIC'], indices_chg['IBOV'])

print("Alfa:", round(res.params[0]*252, 3), "Beta:", round(res.params[1], 3))
print("R2:", round(res.rsquared, 2)) # R2 é o coeficiente de determinação. Quanto maior, mais próximo da reta os valores estão

Alfa: 0.029 Beta: 0.946
R2: 0.69


In [26]:
## Carteira com máxima utilidade - com base em dados históricos

means = dados_chg.mean().to_numpy()
rf = indices_chg['SELIC'].mean()
means_rf = means - rf
means_rf

array([0.00052617, 0.00062159, 0.00058053, 0.00048484, 0.00083566])

In [28]:
# Calcular cov dos retornos e transformar em matriz

dados_chg.cov()
cov_mat = dados_chg.cov().to_numpy()

In [30]:
# Cálculo Z
pesos_hist_z = np.matmul(np.linalg.inv(cov_mat), means_rf)

# normalização para obtenção de pesos ótimos
pesos_hist_final = pesos_hist_z/np.sum(pesos_hist_z)
pesos_hist_final

array([0.05738504, 0.20637801, 0.31664216, 0.16595122, 0.25364357])

In [31]:
port = dados.dot(pesos_hist_final).pct_change()
port.fillna(0, inplace =  True)
port
res = calc_capm(port, indices_chg['SELIC'], indices_chg['IBOV'])

print("Alfa:", round(res.params[0]*252, 3), "Beta:", round(res.params[1], 3))
print("R2:", round(res.rsquared, 2))

Alfa: 0.04 Beta: 0.839
R2: 0.53


In [32]:
dados['PORT1'] = dados.dot(pesos_hist_final)
dados_chg = dados.pct_change().fillna(0)

ret_aa = ((dados.iloc[-1]/dados.iloc[0])**(1/5)) - 1
print("Retorno anualizado:\n", ret_aa)

vol_aa = dados_chg.std()*np.sqrt(252) # quanto menor a vol, melhor
print("Volatilidade anualizada:\n", vol_aa)

dados = dados.drop(['PORT1'], axis = 1)

Retorno anualizado:
 Ticker
BBDC4.SA    0.207330
DIRR3.SA    0.205414
GUAR3.SA    0.218396
ITUB4.SA    0.207749
JBSS3.SA    0.211594
PORT1       0.211633
dtype: float64
Volatilidade anualizada:
 Ticker
BBDC4.SA    0.331644
DIRR3.SA    0.400726
GUAR3.SA    0.344750
ITUB4.SA    0.297338
JBSS3.SA    0.507458
PORT1       0.253384
dtype: float64


In [35]:
# carteira com máxima utilidade com CAPM, retorno ajustado ao risco de mercado

betas = []
for item in range(5):
    vect = [0]*5
    vect[item] = 1
    port = dados.dot(vect).pct_change()
    port.fillna(0, inplace = True)
    res = calc_capm(port, indices_chg['SELIC'], indices_chg['IBOV'])
    print("Alfa:", round(res.params[0]*252, 3), "Beta:", round(res.params[1], 3))
    betas.append(round(res.params[1], 3))

betas

Alfa: 0.021 Beta: 1.273
Alfa: 0.085 Beta: 0.817
Alfa: 0.091 Beta: 0.636
Alfa: 0.021 Beta: 1.153
Alfa: 0.145 Beta: 0.752


[1.273, 0.817, 0.636, 1.153, 0.752]

In [36]:
# retorno estimado
ret_capm = rf + (np.array(betas) * ((indices_chg['IBOV'].mean()) - rf))

# cálculo do z
pesos_capm_z = np.matmul(np.linalg.inv(cov_mat), ret_capm)

# normalização
pesos_capm_final = pesos_capm_z/np.sum(pesos_capm_z)
pesos_capm_final

array([0.11336376, 0.11501044, 0.20934361, 0.44864297, 0.11363922])

In [37]:
port = dados.dot(pesos_capm_final).pct_change()
port.fillna(0, inplace = True)
port

res = calc_capm(port, indices_chg['SELIC'], indices_chg['IBOV'])
print("Alfa:", round(res.params[0]*252, 3), "Beta:", round(res.params[1], 3))
print("R2:", round(res.rsquared, 2))    

Alfa: 0.024 Beta: 0.998
R2: 0.76


In [38]:
dados['PORT1'] = dados.dot(pesos_capm_final)
dados_chg = dados.pct_change().fillna(0)

ret_aa = ((dados.iloc[-1]/dados.iloc[0])**(1/5)) - 1
print("Retorno anualizado:\n", ret_aa)

vol_aa = dados_chg.std()*np.sqrt(252) # quanto menor a vol, melhor
print("Volatilidade anualizada:\n", vol_aa)

dados = dados.drop(['PORT1'], axis = 1)

Retorno anualizado:
 Ticker
BBDC4.SA    0.207330
DIRR3.SA    0.205414
GUAR3.SA    0.218396
ITUB4.SA    0.207749
JBSS3.SA    0.211594
PORT1       0.210133
dtype: float64
Volatilidade anualizada:
 Ticker
BBDC4.SA    0.331644
DIRR3.SA    0.400726
GUAR3.SA    0.344750
ITUB4.SA    0.297338
JBSS3.SA    0.507458
PORT1       0.250460
dtype: float64
